## 1. Imports


In [50]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns;

In [55]:
data_path = '../data/results/test_5_unbalanced_seed42/'
folders= ['dt','gnb','lgr','gbt']
#folders= ['dt','gnb']

## 2. CSVs

In [56]:
def impact_csvs(data_path= 'data/results/',b_or_w = 'Black', folders= ['dt','gnb','lgr','gbt']):

    col_names_eg = []
    col_names_gs = []

    for i,f in enumerate(folders):
        if b_or_w == 'Black':
            path = f'{data_path}{f}/{f}_black_results.csv'
        else:
            path = f'{data_path}{f}/{f}_white_results.csv'

        df = pd.read_csv(path,index_col=0)
        df = df.reset_index()

        col_names_eg.append(f'EG+{f.upper()}')
        col_names_gs.append(f'GS+{f.upper()}')

        if i == 0:
            joined_df = df.iloc[:,-1]
        else:
            joined_df = pd.concat([joined_df, df.iloc[:,-1]], axis=1)

    joined_df.set_axis(folders, axis=1)


    # split dataframe after the two reduction algorithms
    df_eg = joined_df.iloc[:6,:]
    df_gs = pd.concat([joined_df.iloc[0:1,:],joined_df.iloc[6:,:]])

    # set new index
    df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df_eg.set_index('Constraint',inplace=True)
    df_gs['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df_gs.set_index('Constraint',inplace=True)
    df_eg.columns = col_names_eg
    df_gs.columns = col_names_gs

    df_final = pd.concat([df_eg, df_gs], axis=1)
    print('Group: ',b_or_w,'\n DataFrame: \n',df_final)
    print('A')
    df_final.to_csv(f'{data_path}/{b_or_w}_DI.csv')
    print('B')


In [57]:
impact_csvs(data_path,'Black', folders= ['dt','gnb','lgr','gbt'])
impact_csvs(data_path,'White', folders= ['dt','gnb','lgr','gbt'])

Group:  Black 
 DataFrame: 
              EG+DT  EG+GNB  EG+LGR  EG+GBT  GS+DT  GS+GNB  GS+LGR  GS+GBT
Constraint                                                               
Unmitigated   5.55    0.00    7.16    5.64   5.55    0.00    7.16    5.64
DP          -39.09  -38.60  -39.62  -39.58 -53.67    0.00  -53.67  -53.67
EO           -6.94  -10.55   -7.28   -4.09   5.55  -15.26    7.16    5.64
EOO         -17.90  -22.17  -19.66  -17.06 -32.25  -22.62  -30.22  -33.17
FPER         -4.06  -17.61   -6.08   -4.36   5.55    0.00    7.16    5.64
ERP           5.55    7.23    7.33    5.60   5.55    8.41    7.29    5.64
A
B
Group:  White 
 DataFrame: 
              EG+DT  EG+GNB  EG+LGR  EG+GBT  GS+DT  GS+GNB  GS+LGR  GS+GBT
Constraint                                                               
Unmitigated  38.71   36.55   38.22   38.80  38.71   36.55   38.22   38.80
DP           39.08   39.01   38.84   39.08  39.21   36.55   39.24   39.24
EO           36.98   35.95   36.76   37.27  38.71 

C:\Users\HaRi\AppData\Local\Temp\ipykernel_10244\620886726.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\AppData\Local\Temp\ipykernel_10244\620886726.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


## 3. FP/TP/TN/FN Ratios

In [58]:
# Ratios
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts(normalize=True)
    df = df.reset_index()
    df = df.rename(columns= {0:'Ratio'})
    
    df = df.pivot(index='Category', columns='ID')['Ratio']
    df = df.fillna(0)
    df = df.round(decimals = 3)
    print('Classifier: ',f,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}{f}/{f}_type_ratios.csv')

Classifier:  dt 
 DataFrame: 
 ID        egdpB  egdpW  egeoB  egeoW  egerpB  egerpW  egfprpB  egfprpW  \
Category                                                                 
FN        0.012  0.052  0.044  0.084   0.085   0.080    0.048    0.052   
FP        0.442  0.063  0.191  0.064   0.086   0.075    0.169    0.063   
TN        0.220  0.177  0.471  0.176   0.576   0.165    0.493    0.177   
TP        0.327  0.708  0.295  0.676   0.254   0.680    0.291    0.708   

ID        egtprpB  egtprpW  gsdpB  gsdpW  gseoB  gseoW  gserpB  gserpW  \
Category                                                                 
FN          0.029    0.052  0.000  0.081  0.085  0.045   0.085   0.045   
FP          0.272    0.063  0.662  0.048  0.086  0.070   0.086   0.070   
TN          0.389    0.177  0.000  0.192  0.576  0.171   0.576   0.171   
TP          0.309    0.708  0.338  0.678  0.254  0.715   0.254   0.715   

ID        gsfprpB  gsfprpW  gstprpB  gstprpW  testB  testW  unmitB  unmitW  
Ca

In [36]:
# Absolute numbers
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts()
    df = df.reset_index()
    df = df.rename(columns= {0:'Number'})
    
    df = df.pivot(index='Category', columns='ID')['Number']
    df = df.fillna(0)
    print('Classifier: ',f,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}{f}/{f}_type_absolute.csv')

Classifier:  dt 
 DataFrame: 
 ID         egdpB   egdpW   egeoB   egeoW  egerpB  egerpW  egfprpB  egfprpW  \
Category                                                                     
FN         280.0   580.0   475.0   433.0   633.0   368.0    528.0    317.0   
FP        1436.0   510.0   793.0   791.0   608.0  1209.0    735.0    825.0   
TN        2169.0  3095.0  2812.0  2814.0  2997.0  2396.0   2870.0   2780.0   
TP        3325.0  3025.0  3130.0  3172.0  2972.0  3237.0   3077.0   3288.0   

ID        egtprpB  egtprpW   gsdpB   gsdpW   gseoB   gseoW  gserpB  gserpW  \
Category                                                                     
FN          475.0    436.0   457.0   347.0   633.0   203.0   633.0   203.0   
FP          793.0    668.0   854.0   785.0   608.0  1124.0   608.0  1124.0   
TN         2812.0   2937.0  2751.0  2820.0  2997.0  2481.0  2997.0  2481.0   
TP         3130.0   3169.0  3148.0  3258.0  2972.0  3402.0  2972.0  3402.0   

ID        gsfprpB  gsfprpW  gst

# Analyzing Scores

#### Extractig Scores from csv into dataframes

In [37]:
# Scores Data Frames
classifier_dfs = {}
dfs_b = {}
dfs_w = {}
dfs_eg = {}
dfs_gs = {}
for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    df_eg = pd.concat([df.iloc[:,:4],df.filter(like='eg')],axis=1)
    df_gs = pd.concat([df.iloc[:,:4],df.filter(like='gs')],axis=1)
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white
    dfs_eg[f] = df_eg
    dfs_gs[f] = df_gs
print(classifier_dfs['dt'])

      testB  testW  unmitB  unmitW  egdpB  egdpW  egeoB  egeoW  egtprpB  \
0       605    622     680     697    680    697    680    697      680   
1       543    555     393     405    393    555    393    555      393   
2       568    502     643     502    643    502    643    502      643   
3       632    728     707     803    707    803    707    803      707   
4       622    390     472     390    472    390    472    390      472   
...     ...    ...     ...     ...    ...    ...    ...    ...      ...   
7205    622    434     472     434    472    434    472    434      472   
7206    475    483     475     483    475    483    475    483      475   
7207    515    565     515     415    365    565    515    565      515   
7208    524    464     524     464    374    464    374    464      374   
7209    355    456     355     456    355    456    355    456      355   

      egtprpW  egfprpB  egfprpW  egerpB  egerpW  gsdpB  gsdpW  gseoB  gseoW  \
0         697      6

### Checking if normal distributions:

if p < 0.01 (or < 0.05) then the distribution is significantly different from a normal distribution

In [38]:
for c,df in classifier_dfs.items():
    print('Classifier:',c)
    for col in df:
        data=df[col].dropna(axis=0)
        _,p = stats.kstest(data, "norm")
        if p > 0.01:
            print(col,',p:',p)
    print('Check for norm Distributions done')

Classifier: dt
Check for norm Distributions done
Classifier: gnb
Check for norm Distributions done
Classifier: lgr
Check for norm Distributions done
Classifier: gbt
Check for norm Distributions done


### Mann Whitney U test:

“a two-sample rank test for the difference between two population medians . . . It assumes that the data are independent random samples from two populations that have the same shape.”

In [39]:
mwu_path = f'{data_path}mwu/'
os.makedirs(mwu_path,exist_ok = True)

#### Variance of Distributions EG vs. GS
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [40]:
p_vals = pd.DataFrame(data={'Constraints': []})
p_signi = pd.DataFrame(data={'Constraints': []})
    
for (c1,df_eg),(c2,df_gs) in zip(dfs_eg.items(),dfs_gs.items()):
    col_signi = []
    col_vals = []
    idx = []
    
    for col_eg,col_gs in zip(df_eg,df_gs):
        
        idx.append(col_eg[2:])
        
        data_eg=df_eg[col_eg].dropna(axis=0)
        data_gs=df_gs[col_gs].dropna(axis=0)
        
        _,p = stats.mannwhitneyu(data_eg, data_gs)
        
        col_vals.append(p)
        if p< 0.05:
            col_signi.append('s')
        else:
            col_signi.append(' ')
            
    p_signi[c1] = col_signi
    p_vals[c1] = col_vals
    
p_vals['Constraints'] = idx
p_vals = p_vals.set_index('Constraints')
p_signi['Constraints'] = idx
p_signi = p_signi.set_index('Constraints')

p_vals = p_vals.round(decimals=3)
print(p_signi)
    
p_vals.to_csv(f'{mwu_path}p_eg_gs.csv')
p_signi.to_csv(f'{mwu_path}significanz_eg_gs.csv')


            dt gnb lgr gbt
Constraints               
stB                       
stW                       
mitB                      
mitW                      
dpB          s   s   s   s
dpW          s   s       s
eoB          s   s   s   s
eoW          s   s   s   s
tprpB            s        
tprpW            s        
fprpB            s       s
fprpW        s   s   s   s
erpB                      
erpW             s   s    


#### Variance of Distributions Each Model against each other
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [26]:
def p_values_mwu_1model(dfs,model=''):
    p_vals = pd.DataFrame(data={'Classifier': []})
    p_signi = pd.DataFrame(data={'Classifier': []})
    
    for c1,df1 in dfs.items():
        col_signi = []
        col_vals = []
        idx = []
        
        data1 = df1[model].dropna(axis=0)
        
        for c2,df2 in dfs.items():
            idx.append(c2)
            
            data2 =df2[model].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data1, data2)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c1] = col_signi
        p_vals[c1] = col_vals
        
    p_vals['Classifier'] = idx
    p_vals = p_vals.set_index('Classifier')
    p_signi['Classifier'] = idx
    p_signi = p_signi.set_index('Classifier')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_{model}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_{model}.csv')

In [27]:
for col in classifier_dfs['dt']:
    print('\nC:',col)
    p_values_mwu_1model(classifier_dfs,col)


C: testB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: testW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: unmitB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: unmitW
           dt gnb lgr gbt
Classifier               
dt              s        
gnb         s       s   s
lgr             s        
gbt             s        

C: egdpB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: egdpW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C

#### Variance of Distributions unmitigated v Mitigated for each race

if p < 0.001 (or < 0.0005) then the distributions are significantly different from each other

In [28]:
def p_race_mwu(dfs, b_or_w = 'B'):
    p_vals = pd.DataFrame(data={'Constraints': []})
    p_signi = pd.DataFrame(data={'Constraints': []})
    
    for c,df in dfs.items():
        
        c = f'{c}{b_or_w}'
        col_signi = []
        col_vals = []
        idx = []
        
        data_unmiti = df[f'unmit{b_or_w}'].dropna(axis=0)
        
        for col in df:
            
            idx.append(col[:-1])
            
            data_miti=df[col].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data_unmiti, data_miti)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c] = col_signi
        p_vals[c] = col_vals
        
    p_vals['Constraints'] = idx
    p_vals = p_vals.set_index('Constraints')
    
    
    
    p_signi['Constraints'] = idx
    p_signi = p_signi.set_index('Constraints')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_un_vs_miti_{b_or_w}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_un_vs_miti_{b_or_w}.csv')
   

In [29]:
print('Black:')
p_race_mwu(dfs_b,'B')

print('\nWhite:')
p_race_mwu(dfs_w,'W')

Black:
            dtB gnbB lgrB gbtB
Constraints                   
test          s    s    s    s
unmit                         
egdp          s    s    s    s
egeo               s          
egtprp             s    s     
egfprp             s          
egerp                         
gsdp               s    s    s
gseo                          
gstprp             s    s     
gsfprp                        
gserp                         

White:
            dtW gnbW lgrW gbtW
Constraints                   
test          s         s    s
unmit                         
egdp          s    s    s    s
egeo               s         s
egtprp        s    s         s
egfprp        s    s    s    s
egerp                         
gsdp          s    s    s    s
gseo                          
gstprp        s         s    s
gsfprp                        
gserp              s    s     


In [30]:
stats.pearsonr(data_unmiti, data_miti)

NameError: name 'data_unmiti' is not defined